In [44]:
%store -r

In [45]:
import matplotlib.pyplot as plt
import pandas as pd
import rioxarray as rxr
import rioxarray.merge as rxrmerge
import xarray as xr

# Harmonize your data
 Make sure that the grids for each of your layers match up. Check out the ds.rio.reproject_match() method from rioxarray.

In [47]:
maca_da_repr_list = []

for i in range(len(MACAv2_df)):

    maca_da_orig = (
        MACAv2_df.iloc[i]['climate_da']
        .rio.write_crs(4326)
        .rio.set_spatial_dims('lat', 'lon')
        .groupby('time.year')
        .sum()
        .min('year')
    )
    maca_da_repr = maca_da_orig.rio.reproject_match(topo_df.elevation_da[0])
    maca_da_repr_list.append(maca_da_repr)



In [48]:
MACAv2_df['climate_da'] = maca_da_repr_list

In [61]:
soil_da_repr_list = []

for i in range(len(soil_df)):

    soil_da_orig = (
        soil_df.iloc[i]['soil_da'])
    soil_da_repr = soil_da_orig.rio.reproject_match(topo_df.elevation_da[0])
    soil_da_repr_list.append(soil_da_repr)

In [62]:
soil_df['soil_da'] = soil_da_repr_list

In [66]:
slope_da_repr_list = []

for i in range(len(topo_df)):

    slope_da_orig = (
        topo_df.iloc[i]['slope_da'])
    slope_da_repr = slope_da_orig.rio.reproject_match(topo_df.elevation_da[0])
    slope_da_repr_list.append(slope_da_repr)

In [67]:
topo_df['slope_da'] = slope_da_repr_list

In [68]:
# List of DataFrames to loop through
dfs = [soil_df, topo_df, MACAv2_df]

forest_df = dfs[0]

for merging_df in dfs[1:]:
    # Join DataFrames
    forest_df = pd.merge(forest_df, merging_df, on='site_name')

forest_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4 entries, 0 to 3
Data columns (total 9 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   site_name         4 non-null      object
 1   soil_variable     4 non-null      object
 2   soil_da           4 non-null      object
 3   elevation_da      4 non-null      object
 4   slope_da          4 non-null      object
 5   climate_variable  4 non-null      object
 6   climate_scenario  4 non-null      object
 7   start_year        4 non-null      int64 
 8   climate_da        4 non-null      object
dtypes: int64(1), object(8)
memory usage: 420.0+ bytes


In [58]:
forest_df

,site_name,soil_variable,soil_da,elevation_da,slope_da,climate_variable,climate_scenario,start_year,climate_da
0,George Washington National Forest,ph,[[<xarray.DataArray ()> Size: 4B\narray(5.1427...,"[[<xarray.DataArray ()> Size: 4B\narray(394., ...",[[<xarray.DataArray 'slope' ()> Size: 4B\narra...,pr,rcp45,2096,[[[<xarray.DataArray 'precipitation' ()> Size:...
1,George Washington National Forest,ph,[[<xarray.DataArray ()> Size: 4B\narray(5.1427...,"[[<xarray.DataArray ()> Size: 4B\narray(394., ...",[[<xarray.DataArray 'slope' ()> Size: 4B\narra...,pr,rcp85,2096,[[[<xarray.DataArray 'precipitation' ()> Size:...
2,Jefferson National Forest,ph,[[<xarray.DataArray ()> Size: 4B\narray(4.7129...,"[[<xarray.DataArray ()> Size: 4B\narray(366., ...",[[<xarray.DataArray 'slope' ()> Size: 4B\narra...,pr,rcp45,2096,[[[<xarray.DataArray 'precipitation' ()> Size:...
3,Jefferson National Forest,ph,[[<xarray.DataArray ()> Size: 4B\narray(4.7129...,"[[<xarray.DataArray ()> Size: 4B\narray(366., ...",[[<xarray.DataArray 'slope' ()> Size: 4B\narra...,pr,rcp85,2096,[[[<xarray.DataArray 'precipitation' ()> Size:...


In [69]:
%store forest_df

Stored 'forest_df' (DataFrame)


In [49]:
def print_raster(raster):
    print(
        f"shape: {raster.rio.shape}\n"
        f"resolution: {raster.rio.resolution()}\n"
        f"bounds: {raster.rio.bounds()}\n"
        f"sum: {raster.sum().item()}\n"
        f"CRS: {raster.rio.crs}\n"
    )